In [ ]:
# Upgrade Oracle ADS to pick up latest features and maintain compatibility with Oracle Cloud Infrastructure.

!pip install -U oracle-ads

<font color=gray>Oracle Data Science service sample notebook.

Copyright (c) 2023 Oracle, Inc.  All rights reserved.
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

***
# <font color=red>Train, Register, and Deploy HuggingFace Model</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure Data Science Service Team </font></p>

***

## Overview:

The `HuggingFacePipelineModel` class in Accelerated Data Science (ADS) is designed to allow you to rapidly get a HuggingFace pipeline into production. The `.prepare()` method creates the model artifacts that are needed to deploy a functioning model without you having to configure it or write code. However, it does allow you to customize the `score.py` file as needed. Simulate a call to a deployed model with the `.verify()` method. This method calls the `load_model()` and `predict()` functions in the `score.py` file. Using `.verify()` allows you to debug your `score.py` file without having to deploy a model. The `.save()` method pushes your `HuggingFacePipelineModel` and the model artifacts to the model catalog. The `.deploy()` method deploys the model to a REST endpoint for you. Finally, the `.predict()` method allows you to call the endpoint to perform model inference.

Compatible conda pack: [PyTorch](https://docs.oracle.com/en-us/iaas/data-science/using/conda-pytorch-fam.htm) for CPU on Python 3.8 (version 1.0)

### Prerequisites

This notebook requires authorization to work with the OCI Data Science Service. Details can be found [here](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/cli/authentication.html#). For the purposes of this notebook what is important to to know is that resource principals will be used absent api_key authentication.

---

Datasets are provided as a convenience. Datasets are considered third-party content and are not considered materials under your agreement with Oracle.
      
You can access the `orcl_attrition` dataset license [here](https://oss.oracle.com/licenses/upl).


In [ ]:
import ads
import logging
import tempfile
import warnings
import PIL.Image
import requests
import cloudpickle

from transformers import pipeline
from shutil import rmtree
from ads.model import HuggingFacePipelineModel

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.ERROR)
warnings.filterwarnings("ignore")

<a id='intro'></a>
# Introduction

## Authenticate

Authentication to the OCI Data Science service is required. Here we default to resource principals.

In [ ]:
ads.set_auth(auth="resource_principal")

<a id="intro_dataset"></a>
## Download Dataset

In [ ]:

image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"

image = PIL.Image.open(requests.get(image_url, stream=True).raw)
image_bytes = cloudpickle.dumps(image)

## Download a Pretrained Model

In [ ]:
segmenter = pipeline(task="image-segmentation", model="facebook/detr-resnet-50-panoptic", revision="fc15262")
preds = segmenter(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)

preds

## Prepare Conda Pack
You can start with the PyTorch conda pack with slug `pytorch110_p38_cpu_v1`. 
- Run `pip install timm` since image segmentation model requires `timm`. 
- Then use `odsc conda init -b your_bucket_name -n bucket_namespace` to config where to store the published conda pack if you have not done this yet. 
- Lastly, run `odsc conda publish -s pytorch110_p38_cpu_v1`.
- Once it's done, you can find the path of the pusblished conda pack under the Environment Explorer `Published` tab. Refresh the page if you cannot find it.

## Prepare the model
Now paste the conda pack path here. Initiate the `HuggingFacePipelineModel` instance and call `prepare` to generate the artifact.

In [ ]:
conda_pack_path = "<replace with the path of your published conda pack>" # it looks like this "oci://bucket_name@namespace/path"

huggingface_pipeline_model = HuggingFacePipelineModel(
    segmenter, artifact_dir=tempfile.mkdtemp()
)
huggingface_pipeline_model.prepare(inference_conda_env=conda_pack_path, force_overwrite=True)

In [ ]:
huggingface_pipeline_model.summary_status()

## Verify

The verify method invokes the ``predict`` function defined inside ``score.py`` in the artifact_dir

#### PIL.Image
`HuggingFacePipelineModel` class supports images directly. It by default uses `cloudpickle` to serialize and deserialize the input data. You can pass in the image to `.verify` function to test locally if everything works.

In [ ]:
preds = huggingface_pipeline_model.verify(image)["prediction"]
[{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]

In [ ]:
print(type(preds[0]['mask']))
preds[0]['mask'][0][:10]

#### Bytes
If you are not using `HuggingFacePipelineModel` to invoke your model, then you need to pass in either Json Serializable data or bytes to the endpoint since Model Deployment only supports these two types currently. Again, you can use `.verify` to test locally. When `auto_serialize_data=True`, the data will be serialized by the default model input serializer which is `cloudpickle`. Set `auto_serialize_data=False` since the image is already serialized to bytes by cloudpickle. And in `score.py`, the payload will be deserialized back to image by `cloudpickle`. If `auto_serialize_data=True`, the image will be serialized twice and cause a problem.

In [ ]:
preds = huggingface_pipeline_model.verify(image_bytes, auto_serialize_data=False)["prediction"]
[{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]

## Save

In [ ]:
huggingface_pipeline_model.save(display_name="HuggingFace Pipeline Model")

In [ ]:
huggingface_pipeline_model.summary_status()

## Deploy

When the model is in the model catalog, you can use the model's `.deploy()` method to deploy it. This method allows you to specify the attributes of the deployment such as the display name, description, instance type and count, the maximum bandwidth, and logging groups. The next cell deploys the model with the default settings except for the custom bandwidth mbps, display name and logging. The `.deploy()` method returns a `ModelDeployment` object.

In [ ]:
deploy = huggingface_pipeline_model.deploy(
    deployment_bandwidth_mbps = 100,
    wait_for_completion = False,
    display_name = "HuggingFace Pipeline Model For Image Segmentation",
)

In [ ]:
huggingface_pipeline_model.summary_status()

## Predict

After the deployment is active, you can call `predict()` on the model object to send request to the deployed endpoint. 

In [ ]:
preds = huggingface_pipeline_model.predict(image)["prediction"]
[{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]

## Invoke Your Model

In [ ]:
headers = {"Content-Type": "application/octet-stream"} 
endpoint = huggingface_pipeline_model.model_deployment.url + "/predict"

preds = requests.post(endpoint, data=image_bytes, auth=ads.common.auth.default_signer()['signer'], headers=headers).json()
[{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds['prediction']]

<a id='clean_up'></a>
# Clean Up

This notebook created a model deployment and a model. This section cleans up those resources. 

The model deployment must be deleted before the model can be deleted. You use the `.delete_deployment()` method on the `HuggingFacePipelineModel` object to do this.

In [ ]:
delete = huggingface_pipeline_model.delete_deployment(wait_for_completion=True)

After the model deployment has been deleted, the `.summary_status()` method shows that the model has been deleted and that the `predict()` method is not available.

In [ ]:
huggingface_pipeline_model.summary_status()

Use the `.delete()` method to delete the model.

In [ ]:
huggingface_pipeline_model.delete()

The next cell removes the model artifacts that were stored on your local drive.

In [ ]:
rmtree(huggingface_pipeline_model.artifact_dir)

<a id='ref'></a>
# References
- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)
- [Understanding Conda Environments](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)
- [Use Resource Manager to Configure Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/orm-configure-tenancy.htm)
- [`runtime.yaml`](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_runtime_yaml.htm#model_runtime_yaml)
- [`score.py`](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_score_py.htm#model_score_py)
- [Model artifact](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/models_saving_catalog.htm#create-models)